In [1]:
import sys
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    # montar el drive, que es donde tenemos el dataset
    from google.colab import drive 
    drive.mount("/content/drive")
    data_dir = "/content/drive/MyDrive/Colab Notebooks/pandas/"  
    sys.path.append(data_dir)
else:
    import os
    data_dir = "data/"

In [2]:
import pandas as pd
import numpy as np

In [3]:
filename = data_dir + "canarias.csv"
df = pd.read_csv(filename, delimiter="\t", decimal=",")
df

,Isla (excepto La Graciosa),Bandera,Escudo,Municipios,Capital,Área (km²),Población (2022)[22]​
0,El Hierro,NaN,NaN,3,Valverde,26871,11 423
1,Fuerteventura[nota 11]​,NaN,NaN,6,Puerto del Rosario,"1665,74[nota 12]​",120 021
2,Gran Canaria,NaN,NaN,21,Las Palmas de Gran Canaria,156010,853 262
3,La Gomera,NaN,NaN,6,San Sebastián de La Gomera,36976,21 798
4,Lanzarote[nota 13]​,NaN,NaN,7,Arrecife,"888,07[nota 14]​",156 112[nota 15]​
5,La Palma,NaN,NaN,14,Santa Cruz de La Palma,70832,83 439
6,Tenerife,NaN,NaN,31,Santa Cruz de Tenerife,203438,931 646


read_html es una función de pandas que me descarga todas las tablas de una página web que tienen una determinada palabra. Necesito la cabecera headers porque si no el sitio web me devuelve un error 406

In [4]:
import requests

url = 'https://es.wikipedia.org/wiki/Canarias'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text, match='Capital')
len(dfs)

/tmp/ipykernel_6523/4026687738.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(r.text, match='Capital')


2

En la web hay dos tablas que contienen la palabra _Capital_

In [5]:
df = dfs[1]
df

,Isla (excepto La Graciosa),Bandera,Escudo,Municipios,Capital,Área (km²),Población (2022)[22]​
0,El Hierro,NaN,NaN,3,Valverde,26871,11 423
1,Fuerteventura[nota 11]​,NaN,NaN,6,Puerto del Rosario,"1665,74[nota 12]​",120 021
2,Gran Canaria,NaN,NaN,21,Las Palmas de Gran Canaria,156010,853 262
3,La Gomera,NaN,NaN,6,San Sebastián de La Gomera,36976,21 798
4,Lanzarote[nota 13]​,NaN,NaN,7,Arrecife,"888,07[nota 14]​",156 112[nota 15]​
5,La Palma,NaN,NaN,14,Santa Cruz de La Palma,70832,83 439
6,Tenerife,NaN,NaN,31,Santa Cruz de Tenerife,203438,931 646


In [13]:
# df.to_csv('data/canarias.csv', index=False, sep='\t')

1. Tiro las columnas Bandera y Escudo
2. Renombro las columnas
3. En la columna Área sustituyo la coma decimal por un punto y con una expresión regular extraigo sólo los caracteres numéricos
4. Elimino el espacio de los miles en la columna población y elimino las notas
5. En la columna Isla no hay números así que elimino los corchetes y lo que hay en el medio
6. Añado una columna llamada provincia

In [6]:
df = df.drop(columns = ['Bandera', 'Escudo']) # Does not do inplace supression
df.columns = ['Isla', 'Municipios', 'Capital', 'Área', 'Población']
df["Área"] = df["Área"].str.replace(",", ".").str.extract(r'(^[.0-9]+)')
df['Población'] = df['Población'].str.replace('\xa0', '').str.extract(r'(^[.0-9]+)')
df["Isla"] = df["Isla"].str.extract(r'([^\[]*)')
df["Provincia"] = ["SCTF", "LPGC", "LPGC", "SCTF", "LPGC", "SCTF", "SCTF"]
df

,Isla,Municipios,Capital,Área,Población,Provincia
0,El Hierro,3,Valverde,26871,11423,SCTF
1,Fuerteventura,6,Puerto del Rosario,1665.74,120021,LPGC
2,Gran Canaria,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,La Gomera,6,San Sebastián de La Gomera,36976,21798,SCTF
4,Lanzarote,7,Arrecife,888.07,156112,LPGC
5,La Palma,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,31,Santa Cruz de Tenerife,203438,931646,SCTF


Un DataFrame es una lista de Series, una para cada columna

In [8]:
df["Isla"]

,Isla,Municipios
0,El Hierro,3
1,Fuerteventura,6
2,Gran Canaria,21
3,La Gomera,6
4,Lanzarote,7
5,La Palma,14
6,Tenerife,31


reset_index crea un índice autoincremental nuevo para las filas de la matriz. El antiguo índice pasa a ser una columna de la tabla llamada _index_

In [16]:
df = df.reset_index()
df

,index,Isla,Municipios,Capital,Área,Población,Provincia
0,0,El Hierro,3,Valverde,26871,11423,SCTF
1,1,Fuerteventura,6,Puerto del Rosario,1665.74,120021,LPGC
2,2,Gran Canaria,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,3,La Gomera,6,San Sebastián de La Gomera,36976,21798,SCTF
4,4,Lanzarote,7,Arrecife,888.07,156112,LPGC
5,5,La Palma,14,Santa Cruz de La Palma,70832,83439,SCTF
6,6,Tenerife,31,Santa Cruz de Tenerife,203438,931646,SCTF


Establezco la columna Isla como índice de la tabla

In [11]:
df = df.set_index("Isla")
df

,Municipios,Capital,Área,Población,Provincia
Isla,,,,,
El Hierro,3,Valverde,26871,11423,SCTF
Fuerteventura,6,Puerto del Rosario,1665.74,120021,LPGC
Gran Canaria,21,Las Palmas de Gran Canaria,156010,853262,LPGC
La Gomera,6,San Sebastián de La Gomera,36976,21798,SCTF
Lanzarote,7,Arrecife,888.07,156112,LPGC
La Palma,14,Santa Cruz de La Palma,70832,83439,SCTF
Tenerife,31,Santa Cruz de Tenerife,203438,931646,SCTF


loc trabaja con los índices y no con las posiciones

In [26]:
df.loc["Gran Canaria":"La Palma", "Capital":"Población"]

,Municipios,Capital,Área,Población,Provincia
Isla,,,,,
Gran Canaria,21,Las Palmas de Gran Canaria,156010,853262,LPGC


iloc trabaja con las posiciones

In [20]:
df.iloc[0:3, 0:2]

,index,Municipios
Isla,,
El Hierro,0,3
Fuerteventura,1,6
Gran Canaria,2,21


Creamos un nuevo índice y recuperamos la Isla como una columna de la tabla por la izquierda

In [21]:
df = df.reset_index()
df

,Isla,index,Municipios,Capital,Área,Población,Provincia
0,El Hierro,0,3,Valverde,26871,11423,SCTF
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF


df['Municipios'] y df.Municipios son series y la expresión booleana genera otra serie de booleanos 

In [22]:
mascara = df["Municipios"] > 6
mascara

0    False
1    False
2     True
3    False
4     True
5     True
6     True
Name: Municipios, dtype: bool

Estas tres sentencias tienen el mismo resultado, una vista del dataframe obtenida tras un filtrado con una máscara

In [23]:
df[mascara]
df[df["Municipios"] > 6]
df.query("Municipios > 6")

,Isla,index,Municipios,Capital,Área,Población,Provincia
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF


Otra forma de de crear la misma máscara sería df['Provincia'] == 'SCTF'

In [24]:
mascara = df["Provincia"].isin(["SCTF"])
mascara

0     True
1    False
2    False
3     True
4    False
5     True
6     True
Name: Provincia, dtype: bool

Añado una fila con el índice 'pepe'

In [25]:
df.loc['pepe'] = ["ibiza", 5, 3, "asd", "asd", 7 , "????"]
df

,Isla,index,Municipios,Capital,Área,Población,Provincia
0,El Hierro,0,3,Valverde,26871,11423,SCTF
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF
pepe,ibiza,5,3,asd,asd,7,????


Tiro una fila especificando el índice

In [26]:
df = df.drop(index="pepe")
df

,Isla,index,Municipios,Capital,Área,Población,Provincia
0,El Hierro,0,3,Valverde,26871,11423,SCTF
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF


Añado una columna especificando los valores

In [27]:
df["viento"] = [1,2,3,4,5,6,7]
df

,Isla,index,Municipios,Capital,Área,Población,Provincia,viento
0,El Hierro,0,3,Valverde,26871,11423,SCTF,1
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,3
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,4
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,5
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,6
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,7


Tiro una columna

In [28]:
df = df.drop(columns="viento")
df

,Isla,index,Municipios,Capital,Área,Población,Provincia
0,El Hierro,0,3,Valverde,26871,11423,SCTF
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF


In [29]:
df["Fvisita"] = ["1/12/2023","13/1/2023","1/12/2022","1/1/2023","12/12/2023","1/12/2020","1/12/2022",]
df

,Isla,index,Municipios,Capital,Área,Población,Provincia,Fvisita
0,El Hierro,0,3,Valverde,26871,11423,SCTF,1/12/2023
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,13/1/2023
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,1/12/2022
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,1/1/2023
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,12/12/2023
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,1/12/2020
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,1/12/2022


In [30]:
df["Fvisita"]=pd.to_datetime(df["Fvisita"], format='%d/%m/%Y')
df

,Isla,index,Municipios,Capital,Área,Población,Provincia,Fvisita
0,El Hierro,0,3,Valverde,26871,11423,SCTF,2023-12-01
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023-01-13
2,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022-12-01
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023-01-01
4,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,2023-12-12
5,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020-12-01
6,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022-12-01


In [31]:
df["year"] = df["Fvisita"].dt.year
df[df["Fvisita"].dt.month == 1]

,Isla,index,Municipios,Capital,Área,Población,Provincia,Fvisita,year
1,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023-01-13,2023
3,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023-01-01,2023


Se carga el indice actual y lo sustituye

In [32]:
df = df.set_index("Fvisita")
df.loc["2022"]

,Isla,index,Municipios,Capital,Área,Población,Provincia,year
Fvisita,,,,,,,,
2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022
2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022


In [27]:
#df.loc["2020":"2022"]

Hay que ordenarlo previamente porque sino genera un error

In [34]:
df.sort_index().loc["2020":"2022"]

,Isla,index,Municipios,Capital,Área,Población,Provincia,year
Fvisita,,,,,,,,
2020-12-01,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020
2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022
2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022


In [35]:
# Recupera la columna index como índice y Fvisita desaparece
# df = df.set_index('index')
# Crea un nuevo índice y Fvisita vuelve a ser una columna normal
df = df.reset_index()
df

,Fvisita,Isla,index,Municipios,Capital,Área,Población,Provincia,year
0,2023-12-01,El Hierro,0,3,Valverde,26871,11423,SCTF,2023
1,2023-01-13,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023
2,2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022
3,2023-01-01,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023
4,2023-12-12,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,2023
5,2020-12-01,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020
6,2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022


In [36]:
mask = pd.to_datetime("today").year - df["Fvisita"].dt.year < 5
df.loc[mask, "Vreciente"] = "Hace poco"
mask = ~mask
df.loc[mask, "Vreciente"] = "Hace bastante"
df

,Fvisita,Isla,index,Municipios,Capital,Área,Población,Provincia,year,Vreciente
0,2023-12-01,El Hierro,0,3,Valverde,26871,11423,SCTF,2023,Hace poco
1,2023-01-13,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
3,2023-01-01,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023,Hace poco
4,2023-12-12,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
5,2020-12-01,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020,Hace bastante
6,2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco


np.select recibe una matriz máscaras y un vector de valores y devuelve un vector cuyos valores se seleccionan de entre los especificados en el vector de valores según lo que indica la matriz de máscaras:
1. Una matriz de booleanos con una fila por elemento del vector de valores y tantas columnas como el vector de salida, que indica en que posiciones seleccionamos cada valor
2. Un vector de valores

In [37]:
df = df.drop("Vreciente", axis=1)
mask1 = pd.to_datetime("today").year - df["Fvisita"].dt.year < 5
mask2 = ~mask1
df["Vreciente"] = np.select([mask1, mask2], ["Hace poco", "Hace bastante"])
df

,Fvisita,Isla,index,Municipios,Capital,Área,Población,Provincia,year,Vreciente
0,2023-12-01,El Hierro,0,3,Valverde,26871,11423,SCTF,2023,Hace poco
1,2023-01-13,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
3,2023-01-01,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023,Hace poco
4,2023-12-12,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
5,2020-12-01,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020,Hace bastante
6,2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco


Las columna con tipo de dato categórico consumen menos memoria pero eso solo se nota cuando las tablas son muy grandes. Creamos una nueva columna de tipo categórico llamada VrecienteCat a partir de las cadenas existentes en la columna Vreciente. Observa que no es una operación in-place. astype sirve para cambiar el tipo de una columna.

In [28]:
from pandas.api.types import CategoricalDtype
df["VrecienteCat"]=df["Vreciente"].astype("category")
df.memory_usage()

Index           132
Fvisita          56
Isla             56
index            56
Municipios       56
Capital          56
Área             56
Población        56
Provincia        56
year             28
Vreciente        56
VrecienteCat    131
dtype: int64

Podemos crear el tipo de dato categórico especificando los valores que puede tener. Cuando creamos una nueva serie con este tipo de dato a partir de los valores de una columna preexistente, las celdas que no entren en ninguna categoría se les imputa el valor nulo <NaN>

In [325]:
visita_cat = CategoricalDtype(categories = ['Nunca', 'Hace poco'], ordered=True)
df["Vreciente"].astype(visita_cat)

0    Hace poco
1    Hace poco
2    Hace poco
3    Hace poco
4    Hace poco
5          NaN
6    Hace poco
Name: Vreciente, dtype: category
Categories (2, object): ['Nunca' < 'Hace poco']

La función cut crea una nueva columna categórica discretizando con 3 intervalos de la misma longitud.

In [326]:
df['Población'] = df['Población'].astype(int)
pd.cut(df["Población"], 3)

0    (10502.777, 318164.0]
1    (10502.777, 318164.0]
2     (624905.0, 931646.0]
3    (10502.777, 318164.0]
4    (10502.777, 318164.0]
5    (10502.777, 318164.0]
6     (624905.0, 931646.0]
Name: Población, dtype: category
Categories (3, interval[float64, right]): [(10502.777, 318164.0] < (318164.0, 624905.0] < (624905.0, 931646.0]]

Si no le asignamos etiquetas a cada categoría usa los intervalos como etiquetas.

In [327]:
pd.cut(df["Población"], 3, labels=["Poca", "Media", "Bastante"])

0        Poca
1        Poca
2    Bastante
3        Poca
4        Poca
5        Poca
6    Bastante
Name: Población, dtype: category
Categories (3, object): ['Poca' < 'Media' < 'Bastante']

Ordenamos primero por Provincia y luego por población de menor a mayor valor. Fíjate que si no lo especificamos no es una operación in-place y por tanto crea un nuevo dataframe

In [328]:
df.sort_values(by=["Provincia" , "Población" ])

,Fvisita,Isla,index,Municipios,Capital,Área,Población,Provincia,year,Vreciente
1,2023-01-13,Fuerteventura,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
4,2023-12-12,Lanzarote,4,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
2,2022-12-01,Gran Canaria,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
0,2023-12-01,El Hierro,0,3,Valverde,26871,11423,SCTF,2023,Hace poco
3,2023-01-01,La Gomera,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023,Hace poco
5,2020-12-01,La Palma,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020,Hace bastante
6,2022-12-01,Tenerife,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco


idxmax devuelve una serie con el índice de la fila con el valor máximo para cada columna

In [329]:
df = df.set_index("Isla")
df["Población"].idxmax()

'Tenerife'

In [330]:
print(df.loc[df["Población"].idxmax()])
df = df.reset_index()
df

Fvisita          2022-12-01 00:00:00
index                              6
Municipios                        31
Capital       Santa Cruz de Tenerife
Área                          203438
Población                     931646
Provincia                       SCTF
year                            2022
Vreciente                  Hace poco
Name: Tenerife, dtype: object


,Isla,Fvisita,index,Municipios,Capital,Área,Población,Provincia,year,Vreciente
0,El Hierro,2023-12-01,0,3,Valverde,26871,11423,SCTF,2023,Hace poco
1,Fuerteventura,2023-01-13,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,Gran Canaria,2022-12-01,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
3,La Gomera,2023-01-01,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023,Hace poco
4,Lanzarote,2023-12-12,4,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
5,La Palma,2020-12-01,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020,Hace bastante
6,Tenerife,2022-12-01,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco


Como ya hay una columna con el nombre 'index' se crea la columna level_0 y si volvemos a ejecutar esta celda se genera un error

In [331]:
df = df.reset_index()
df

,level_0,Isla,Fvisita,index,Municipios,Capital,Área,Población,Provincia,year,Vreciente
0,0,El Hierro,2023-12-01,0,3,Valverde,26871,11423,SCTF,2023,Hace poco
1,1,Fuerteventura,2023-01-13,1,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,2,Gran Canaria,2022-12-01,2,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
3,3,La Gomera,2023-01-01,3,6,San Sebastián de La Gomera,36976,21798,SCTF,2023,Hace poco
4,4,Lanzarote,2023-12-12,4,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
5,5,La Palma,2020-12-01,5,14,Santa Cruz de La Palma,70832,83439,SCTF,2020,Hace bastante
6,6,Tenerife,2022-12-01,6,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco


In [332]:
df = df.drop(columns=["level_0", "index"])
df["Población"].nlargest(3)
df.nlargest(3, "Población")

,Isla,Fvisita,Municipios,Capital,Área,Población,Provincia,year,Vreciente
6,Tenerife,2022-12-01,31,Santa Cruz de Tenerife,203438,931646,SCTF,2022,Hace poco
2,Gran Canaria,2022-12-01,21,Las Palmas de Gran Canaria,156010,853262,LPGC,2022,Hace poco
4,Lanzarote,2023-12-12,7,Arrecife,888.07,156112,LPGC,2023,Hace poco


In [333]:
# esto es una serie de panda
df['Área'] = df['Área'].astype(float)
df["Población"] / df["Área"]
(df["Población"] / df["Área"]).nlargest(3).index
df.iloc[(df["Población"] / df["Área"]).nlargest(3).index]

,Isla,Fvisita,Municipios,Capital,Área,Población,Provincia,year,Vreciente
4,Lanzarote,2023-12-12,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
1,Fuerteventura,2023-01-13,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,Gran Canaria,2022-12-01,21,Las Palmas de Gran Canaria,156010.00,853262,LPGC,2022,Hace poco


In [334]:
#también valdría
idx = (df["Población"] / df["Área"]).sort_values(ascending=False).index
df.iloc[idx[:3]]

,Isla,Fvisita,Municipios,Capital,Área,Población,Provincia,year,Vreciente
4,Lanzarote,2023-12-12,7,Arrecife,888.07,156112,LPGC,2023,Hace poco
1,Fuerteventura,2023-01-13,6,Puerto del Rosario,1665.74,120021,LPGC,2023,Hace poco
2,Gran Canaria,2022-12-01,21,Las Palmas de Gran Canaria,156010.00,853262,LPGC,2022,Hace poco


Como 'int64' no tiene el valor np.nan fuerza la converión de la segunda columna a un 'float64'

In [335]:
data = [[1, 2,10], [3, np.nan, 11], [5, 6, 12]]
df = pd.DataFrame(data, columns=[ "A" , "B", "C" ])
print(df)

   A    B   C
0  1  2.0  10
1  3  NaN  11
2  5  6.0  12


In [336]:
df.isna()

,A,B,C
0,False,False,False
1,False,True,False
2,False,False,False


In [337]:
df.dropna()

,A,B,C
0,1,2.0,10
2,5,6.0,12


In [338]:
df.dropna(axis = 1)

,A,C
0,1,10
1,3,11
2,5,12


In [339]:
df.fillna(1000)

,A,B,C
0,1,2.0,10
1,3,1000.0,11
2,5,6.0,12


In [340]:
df.interpolate(axis = 1)

,A,B,C
0,1.0,2.0,10.0
1,3.0,7.0,11.0
2,5.0,6.0,12.0


In [341]:
df.interpolate()

,A,B,C
0,1,2.0,10
1,3,4.0,11
2,5,6.0,12


In [342]:
df.index = [100,102,101]
df.sort_index()

,A,B,C
100,1,2.0,10
101,5,6.0,12
102,3,NaN,11


In [343]:
df.sort_index().interpolate()

,A,B,C
100,1,2.0,10
101,5,6.0,12
102,3,6.0,11


In [344]:
df

,A,B,C
100,1,2.0,10
102,3,NaN,11
101,5,6.0,12
